In [2]:
import numpy as np
import tvm 
from tvm import te

def broadcast_add(shape1, shape2):
    assert len(shape1) == 2 and len(shape2) == 2, \
        "broadcast tensors should both be 2-dimsion"
    for i in range(len(shape1)):
        assert shape1[i] == shape2[i] or shape1[i] == 1 or shape2[i] == 1, \
            "tensor shapes do not fit for broadcasting"
    A = te.placeholder(shape1, name='A')
    B = te.placeholder(shape2, name='B')
    m = shape1[0] if shape2[0] == 1 else shape2[0]
    n = shape1[1] if shape2[1] == 1 else shape2[1]
    f = lambda x, y: A[0 if shape1[0]==1 else x, 0 if shape1[1]==1 else y] + \
        B[0 if shape2[0]==1 else x, 0 if shape2[1]==1 else y]
    C = te.compute((m, n), f, name='C')
    return A, B, C


In [3]:
m = 3
n = 4
shape1 = (m, 1)
shape2 = (m, n)
A, B, C = broadcast_add(shape1, shape2)
s = te.create_schedule(C.op)
print(tvm.lower(s, [A, B], simple_mode=True))
mod = tvm.build(s, [A, B, C])

@main = primfn(A_1: handle, B_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [3], []),
             B: Buffer(B_2: Pointer(float32), float32, [12], [])}
  buffer_map = {A_1: A, B_1: B}
  preflattened_buffer_map = {A_1: A_3: Buffer(A_2, float32, [3, 1], []), B_1: B_3: Buffer(B_2, float32, [3, 4], [])} {
  allocate(C: Pointer(global float32), float32, [12]), storage_scope = global;
  for (x: int32, 0, 3) {
    for (y: int32, 0, 4) {
      let cse_var_1: int32 = ((x*4) + y)
      C_1: Buffer(C, float32, [12], [], align=32)[cse_var_1] = (A[x] + B[cse_var_1])
    }
  }
}


